In [10]:
import numpy as np
import random

num_entities = 10
num_validators = 500000
#num_slots = 300000
num_slots = 20000
fixed_reward = 0.025
mean_mev_reward = 0.1
sigma_mev_reward = 1
split_threshold = 32

# Create validator_entity_mapping and initialize rewards
validator_entity_mapping = [i % num_entities for i in range(num_validators)]
validator_rewards = np.zeros(num_validators)
entity_rewards = np.zeros(num_entities)

# Create lists of validators per entity
validators_per_entity = [[] for _ in range(num_entities)]
for validator_idx, entity in enumerate(validator_entity_mapping):
    validators_per_entity[entity].append(validator_idx)

# create distribution
num_points_lower = 175_000
num_points_upper = 35_000
max_values = [692, 550, 198]

# Generate the lower part of the distribution (0 to 0.13 ETH)
lower_part = np.random.lognormal(mean=np.log(0.065), sigma=0.5, size=num_points_lower)
lower_part = lower_part[(lower_part >= 0) & (lower_part <= 0.13)]

# Generate the upper part of the distribution (0.13 ETH to max values)
upper_parts = []
for max_value in max_values:
    upper_part = np.random.lognormal(mean=np.log(max_value/3), sigma=1.0, size=num_points_upper // len(max_values))
    upper_part = upper_part[(upper_part > 0.13) & (upper_part <= max_value)]
    upper_parts.append(upper_part)

upper_part = np.concatenate(upper_parts)

# Combine both parts of the distribution
dist = np.concatenate([lower_part, upper_part])

# Run the simulation
for i in range(num_slots):
    # Select a random validator
    if i%100 == 0:
        print(i)
    chosen_validator = random.randrange(num_validators)

    # Assign rewards
    block_reward = fixed_reward
    mev_reward = np.random.choice(dist)
    #print(mev_reward)
    total_reward = block_reward + mev_reward

    validator_rewards[chosen_validator] += total_reward
    entity_rewards[validator_entity_mapping[chosen_validator]] += total_reward

    # Check if we need to spawn a new validator for any entity
    for entity_idx in range(num_entities):
        if entity_rewards[entity_idx] >= split_threshold:
            # Spawn a new validator for this entity
            num_validators += 1
            validator_entity_mapping.append(entity_idx)
            validator_rewards = np.append(validator_rewards, 0)
            validators_per_entity[entity_idx].append(num_validators - 1)

            # Deduct the rewards used to spawn the new validator from the existing validators
            to_deduct = split_threshold
            for validator_idx in validators_per_entity[entity_idx]:
                if to_deduct > 0:
                    deduct_amount = min(validator_rewards[validator_idx], to_deduct)
                    validator_rewards[validator_idx] -= deduct_amount
                    entity_rewards[entity_idx] -= deduct_amount
                    to_deduct -= deduct_amount

# Print the top and bottom 20 validator stakes
sorted_indices = np.argsort(validator_rewards)
print("Top 20 validator stakes:")
for idx in sorted_indices[-20:]:
    print(f"Validator {idx}: {validator_rewards[idx]}")

print("Bottom 20 validator stakes:")
for idx in sorted_indices[:20]:
    print(f"Validator {idx}: {validator_rewards[idx]}")

# Count validators per entity
validators_count_per_entity = [len(validators) for validators in validators_per_entity]

# Calculate the total value each entity controls
entity_values = np.array(validators_count_per_entity) * 32 + entity_rewards

# Print the results
print("Validators per entity:")
for i in range(num_entities):
    print(f"Entity {i}: {validators_count_per_entity[i]}")

print("Total value each entity controls:")
for i in range(num_entities):
    print(f"Entity {i}: {entity_values[i]}")




0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18